# HyperDrive demo notebook

This notebook will demostrate the use of the AzureML tool HyperDrive which allows for distributed hyperparameter tuning.

 Hyperparameters are used to control the training of a machine learning model and hyperparameter tuning is the problem of choosing a set of optimal hyperparameters for a learning algorithm for the specific application. This requires compiling and training a model over and over with different combinations of hyperparameters in a defined hyperparameter space, so the process can be very time intensive. Parallelising this processing can significantly increase the time efficiency of this process. 

In [26]:
%matplotlib inline

In [27]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [29]:
import prd_pipeline

## Set up azure experiment


In [30]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Environment
from azureml.core import Experiment, ComputeTarget, ScriptRunConfig

In [31]:
prd_ws = Workspace.from_config()

In [1]:
use_full_dataset = False
if use_full_dataset:
    azure_dataset_name ='prd_merged_all_events_files'
else:
    #  use subset for development.
    azure_dataset_name ='prd_merged_202110_nswws_amber_oct_files'

In [32]:

azure_experiment_name='prd_mlops_test'
azure_env_name = 'prd_ml_cluster'
cluster_name = 'mlops-test'

In [33]:
prd_model_name = 'azml_cluster_demo_20220414'

In [34]:
target_parameter = 'rainfall_rate_composite'
profile_features = ['air_temperature', 'relative_humidity']
single_lvl_features = ['air_pressure_at_sea_level'] 

In [35]:
prd_exp = Experiment(workspace=prd_ws, name=azure_experiment_name)
prd_exp

Name,Workspace,Report Page,Docs Page
prd_mlops_test,precip_rediagnosis,Link to Azure Machine Learning studio,Link to Documentation


Get the AzML environment (basically a conda environment) from the workspace.

In [41]:
test_run = azureml.core.get_run(prd_exp, 'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa')

In [42]:
test_run.get_best_run_by_primary_metric()

Experiment,Id,Type,Status,Details Page,Docs Page
prd_mlops_test,HD_18fd3ffb-6021-4845-88d0-e706c643a9fa_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
prd_env = Environment.get(workspace=prd_ws, name=azure_env_name)
prd_env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04",
        "baseImageRegistry": {
            "address": "mcr.microsoft.com",
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {},
    "inferencingStackVersion": null,
    "name": "prd_ml_cluster",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            ],
            "dependencies": [
                "python=3.8",

### Load data

Use prd_pipeline to load and preprocess data

In [12]:
# import importlib 
# importlib.reload(prd_cluster_train_demo)

In [13]:
input_data = prd_pipeline.load_data(
    prd_ws,
    dataset_name=azure_dataset_name
)
data_splits, data_dims = prd_pipeline.preprocess_data(
    input_data,
    test_fraction=0.2,
    feature_dict={'profile': profile_features, 'single_level': single_lvl_features,'target': target_parameter,},
    test_savefn='tmp.csv',
)


{'profile': ['air_temperature', 'relative_humidity'], 'single_level': ['air_pressure_at_sea_level'], 'target': 'rainfall_rate_composite'}


In [14]:
# these are example calls to the code for easier debugging than running on a separate cluster
# model = prd_cluster_train_demo.build_model(**data_dims)
# model = prd_cluster_train_demo.train_model(model, data_splits)

In [15]:
import datetime
log_dir = 'log/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

### Execute our training run on a cluster with hyperdrive for parallelised hyperparameter tuning

In [16]:
from azureml.train.hyperdrive import GridParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

In [17]:
prd_demo_compute_target = ComputeTarget(workspace=prd_ws, name=cluster_name)
prd_demo_compute_target

AmlCompute(workspace=Workspace.create(name='precip_rediagnosis', subscription_id='07efdc52-cd27-48ed-9443-3aad2b6b777b', resource_group='precip_rediagnosis'), name=mlops-test, id=/subscriptions/07efdc52-cd27-48ed-9443-3aad2b6b777b/resourceGroups/precip_rediagnosis/providers/Microsoft.MachineLearningServices/workspaces/precip_rediagnosis/computes/mlops-test, type=AmlCompute, provisioning_state=Succeeded, location=uksouth, tags={})

Hyperparameters that we want to vary using hyperdrive need to be input arguments for the prd_cluster_train_demo.py script which is called through ScriptRunConfig. Hyperparameters set in prd_demo_args will be overwritten by Hyperdrive. 

In [18]:
nepochs = 1

In [19]:
prd_demo_args = ['--dataset-name', azure_dataset_name,
                 '--target-parameter', target_parameter,
                 '--model-name', prd_model_name,
                ]

prd_demo_args += ['--profile-features']
prd_demo_args += profile_features
prd_demo_args += ['--single-level_features']
prd_demo_args += single_lvl_features
prd_demo_args += ['--epochs', nepochs]
prd_demo_args += ['--batch-size', 128]
prd_demo_args += ['--learning-rate', 0.01]

prd_demo_args

['--dataset-name',
 'sd3',
 '--target-parameter',
 'rainfall_rate_composite',
 '--model-name',
 'azml_cluster_demo_20220414',
 '--profile-features',
 'air_temperature',
 'relative_humidity',
 '--single-level_features',
 'air_pressure_at_sea_level',
 '--epochs',
 1,
 '--batch-size',
 128,
 '--learning-rate',
 0.01]

In [20]:
prd_run_src = ScriptRunConfig(source_directory=os.getcwd(),
                      script='prd_cluster_train_demo.py',
                      arguments=prd_demo_args,
                      compute_target=prd_demo_compute_target,
                      environment=prd_env)

### HyperDrive configuration

The next step is to configure our HyperDrive run. The run config defined above is passed to our HyperDriveConfig as the run_config. We must provide a <code>hyperparameter_sampling</code> explained in more detail below. We are required to also provide the <code>primary_metric_name</code> (either be the models loss function or a metric define when compiling the model) and the <code>primary_metric_goal</code> to either minimize or maximize this metric. We must also define <code>max_total_runs</code> (the upper bound of the number of runs, may be smaller depending on the defined hyperparameter space and sampling strategy) and <code>max_concurrent_runs</code>, if this is set to None all run are launched in parallel.
There is also the option to define an early stopping policy with the <code>policy</code> argument.

#### Hyperparameter sampling

There are three different classes for sampling the hyperparameter space: 
- <code>GridParameterSampling</code>: define a search space as a grid of hyperparameter based on the given hyperparameter space, then evaluates every position in the grid in order (note if max_total_runs < total potential combinations, then this will only run a subsample of the grid).
- <code>RandomParameterSampling</code>: randomly samples hyperparameter combinations from the hyperparameter space 
- <code>BayesianParameterSampling</code>: defines Bayesian sampling over a hyperparameter space, tries to intelligently pick the next sample of hyperparameters based on how the previous samples performed.

We then define the hyperparameter space from which to select the combination of hyperparameters to assess. The hyperparameters that we want to tune should be input arguments to the script which is being run by ScriptRunConfig and any hyperparameter arguments that are input will be overwritten by hyperdrive.

In AzureML, there are different ways to define the set to sample each hyperparameters from (<code>choice</code>, <code>lognormal</code>, <code>loguniform</code>, <code>normal</code>, <code>qlognormal</code>, <code>qloguniform</code>, <code>qnormal</code>, <code>quniform</code>, <code>randint</code> and <code>uniform</code>).

In [21]:
ps = RandomParameterSampling(
    {
        '--batch-size': choice(32, 64, 128),
        '--learning-rate': loguniform(-6, -1)
    }
)

#### Early stopping policy

We can define an early stopping policy in which means that any poorly performing experiment runs are canceled and new ones started. Here we use BanditPolicy with a slack criteria of 0.1 (the ratio of slack allowed with respect to the best performing training run) and evaluation internal of 2 (the frequency for applying the policy, here every two training steps).

In [22]:
early_stop_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [23]:
htc = HyperDriveConfig(run_config=prd_run_src, 
                       hyperparameter_sampling=ps, 
                       policy=early_stop_policy, 
                       primary_metric_name='mean_absolute_error', 
                       primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                       max_total_runs=4,
                       max_concurrent_runs=4)

### HyperDrive running

In [24]:
prd_run = prd_exp.submit(htc)
prd_run

Experiment,Id,Type,Status,Details Page,Docs Page
prd_mlops_test,HD_18fd3ffb-6021-4845-88d0-e706c643a9fa,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
prd_run.wait_for_completion()
assert(prd_run.get_status() == "Completed")

### HyperDrive results

We can then select the model which performs best against the selected primary metric, as defined within the HyperDriveConfig.

In [ ]:
prd_best_run = prd_run.get_best_run_by_primary_metric()
prd_best_run

We can also return information from each of the different hyperdrive child run

In [ ]:
prd_run.get_children_sorted_by_primary_metric()

In [ ]:
prd_run.get_metrics()

In [44]:
test_run.get_metrics()

{'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa_0': {'mean_absolute_error': 14.027489237225254,
  'R-squared score': 0.586384162867827},
 'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa_1': {'mean_absolute_error': 14.818366171589505,
  'R-squared score': 0.49815163454629796},
 'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa_3': {'mean_absolute_error': 14.964776840180555,
  'R-squared score': 0.5118016772403515},
 'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa_2': {'mean_absolute_error': 14.473514401967618,
  'R-squared score': 0.5935323290592469}}

In [ ]:
prd_run.get_hyperparameters()

In [45]:
test_run.get_hyperparameters()

{'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa_0': '{"--batch-size": 32, "--learning-rate": 0.001}',
 'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa_1': '{"--batch-size": 32, "--learning-rate": 0.01}',
 'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa_2': '{"--batch-size": 64, "--learning-rate": 0.001}',
 'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa_3': '{"--batch-size": 64, "--learning-rate": 0.01}'}